This is the data ingestion & exploration notebook. Please take note that the Explore_Amazon_Data module will read in and modify existing tables if you give it an existing table name.

In [ ]:
import pandas as pd
import os 
import gzip
import sqlite3 as sq
import Explore_Amazon_Data as amzn  #This is our data ingestion module, please ensure that this python file is in the same folder as the Notebook.
import matplotlib as mpl

The two cells below do two things: 
1.) Defines the three inputs into the Explore_Amazon_Data module. amazon_file_path is the location of the raw aws customer review data. db_file_path is the location of the SQLite DB. table_name defines which table the data will be dumped into.

2.) Run the ingestion module to read in raw files and dump to the specified tables in the SQLite database.

In [ ]:
#amazon_file_path = #file path of aws customer review data goes here
#db_file_path = #file path of SQLite database
#table_name = #table name to store data in SQLite database

In [ ]:
#amzn.create(amazon_file_path, db_file_path, table_name)

The below cell is the beginning of the data exploration. It sets up a connection and a cursor. You can then write SQL queries (as shown below) to pull data from the DB.

In [ ]:
conn = sq.connect('C:/Users/arjun/Desktop/amazon.db') #sqliteDB path goes in parantheses
crsr = conn.cursor()

df = pd.read_sql_query('''
                SELECT DISTINCT *
                from amazon_music_reviews LIMIT 1''', conn)

print(df)

DATA COLUMNS & DESCRIPTIONS:

marketplace       - 2 letter country code of the marketplace where the review was written.
customer_id       - Random identifier that can be used to aggregate reviews written by a single author.
review_id         - The unique ID of the review.
product_id        - The unique Product ID the review pertains to. In the multilingual dataset the reviews
                    for the same product in different countries can be grouped by the same product_id.
product_parent    - Random identifier that can be used to aggregate reviews for the same product.
product_title     - Title of the product.
product_category  - Broad product category that can be used to group reviews 
                    (also used to group the dataset into coherent parts).
star_rating       - The 1-5 star rating of the review.
helpful_votes     - Number of helpful votes.
total_votes       - Number of total votes the review received.
vine              - Review was written as part of the Vine program.
verified_purchase - The review is on a verified purchase.
review_headline   - The title of the review.
review_body       - The review text.
review_date       - The date the review was written.

In [ ]:
#This cell checks for the count of NULLs per column. Highest NULL count is 202 NULL values for column review_headline.

null_counts = pd.read_sql_query('''
                        SELECT 
                            COUNT(*) - COUNT(marketplace) as marketplace_null_count,
                            COUNT(*) - COUNT(customer_id) as customer_id_null_count,
                            COUNT(*) - COUNT(review_id) as review_id_null_count,
                            COUNT(*) - COUNT(product_id) as product_id_null_count,
                            COUNT(*) - COUNT(product_parent) as product_parent_null_count,
                            COUNT(*) - COUNT(product_title) as product_title_null_count,
                            COUNT(*) - COUNT(product_category) as product_category_null_count,
                            COUNT(*) - COUNT(star_rating) as star_rating_null_count,
                            COUNT(*) - COUNT(helpful_votes) as helpful_votes_null_count,
                            COUNT(*) - COUNT(total_votes) as total_votes_null_count,
                            COUNT(*) - COUNT(vine) as vine_null_count,
                            COUNT(*) - COUNT(verified_purchase) as verified_purchase_null_count,
                            COUNT(*) - COUNT(review_headline) as review_headline_null_count
                        FROM
                            amazon_music_reviews
                        ''', conn)

print(null_counts)

In [ ]:
star_rating_df = pd.read_sql_query('''
                                    SELECT 
                                        star_rating
                                    FROM
                                        amazon_music_reviews
                                    WHERE
                                        star_rating IS NOT NULL
                                    ''', conn
                                    )

check = pd.to_numeric(star_rating_df.star_rating, errors='ignore')

check.hist()